# Imports

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import json
import time
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import numpy as np

## Init Browser

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\kheat\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


# Web Scrape Loop

In [3]:
years = list(np.arange(2016, 2022))


for year in years:
    clean_names = []
    seeds = []
    
    df = pd.DataFrame()
    
    url = f'https://kenpom.com/index.php?y={year}'
    browser.visit(url)
    time.sleep(0.5)
    
    html=browser.html
    soup = bs(html, 'lxml')
    
    dfs = pd.read_html(browser.html)
    df = dfs[0]
    df.columns = [x[1] for x in df.columns]
    df.Team[0][-2:]
    str.isnumeric(df.Team[0][-2:].strip())
    
    for _, row in df.iterrows():
        seed = row.Team[-2:].strip()

        if str.isnumeric(seed):
            clean_name = row.Team[:-2]

        else:
            clean_name = row.Team
            seed = ''

        clean_names.append(clean_name)
        seeds.append(seed)
  
    df["TeamName_Clean"] = clean_names
    df["Seed"] = seeds
    
    df.columns = ['Rk', 'Team', 'Conf', 'W-L', 'AdjEM', 'AdjO', 'AdjO_Rank', 'AdjD', 'AdjD_Rank',
       'AdjT', 'AdjT_Rank', 'Luck', 'Luck_Rank', 'SOS_AdjEM', 'AdjEM_Rank', 'SOS_OppO', 'OppO_Rank',
       'SOS_OppD', 'OppD_Rank', 'NCSOS_AdjEM', 'AdjEM_Rank', 'TeamName_Clean', 'Seed']

    df_kenPom = df.drop(['Team','W-L','AdjO_Rank', 'AdjD_Rank', 'AdjT_Rank','Luck_Rank','AdjEM_Rank', 
                         'OppO_Rank', 'OppD_Rank','AdjEM_Rank', 'Seed'], axis=1)

    df_kenPom = df_kenPom[['Rk', 'TeamName_Clean', 'AdjEM', 'AdjO', 'AdjD', 'AdjT', 'Luck', 'SOS_AdjEM', 
                           'SOS_OppO', 'SOS_OppD', 'NCSOS_AdjEM']]
    
    df_kenPom['Year'] = int(year)

    # Write each df to CSV
    df_kenPom.to_csv(f'Data/kenPom_{year}.csv', index=False)


# 2020 Data Cleaning

In [4]:
# Since the 2020 season was cancelled, final standings were reported differently compared to other years

df_2020 = pd.read_csv('Data/kenPom_2020.csv')
df_2020

,Rk,TeamName_Clean,AdjEM,AdjO,AdjD,AdjT,Luck,SOS_AdjEM,SOS_OppO,SOS_OppD,NCSOS_AdjEM,Year
0,1,Kansas 1*,30.23,115.8,85.5,67.3,0.040,12.66,107.4,94.7,9.58,2020
1,2,Gonzaga 1*,26.95,121.3,94.4,71.9,0.050,2.42,103.5,101.0,-2.09,2020
2,3,Baylor 1*,25.49,113.5,88.1,66.2,0.016,10.20,106.4,96.2,1.38,2020
3,4,Dayton 1*,24.93,119.1,94.1,67.6,0.002,2.74,104.1,101.3,-0.74,2020
4,5,Duke 3*,24.62,115.7,91.1,72.0,-0.009,7.28,106.0,98.7,2.60,2020
...,...,...,...,...,...,...,...,...,...,...,...,...
348,349,Maryland Eastern Shore,-26.14,81.7,107.8,67.5,-0.024,-6.40,97.6,104.1,7.78,2020
349,350,Howard,-26.15,93.6,119.7,70.2,-0.015,-8.29,96.7,105.0,0.96,2020
350,351,Mississippi Valley St.,-27.87,89.8,117.6,77.1,-0.027,-6.01,97.8,103.9,5.14,2020
351,352,Kennesaw St.,-28.71,83.5,112.2,68.2,-0.042,-1.74,102.0,103.7,4.10,2020


In [5]:
df_2020['TeamName_Clean'] = df_2020['TeamName_Clean'].str.strip('*1234567890')
df_2020.head()

,Rk,TeamName_Clean,AdjEM,AdjO,AdjD,AdjT,Luck,SOS_AdjEM,SOS_OppO,SOS_OppD,NCSOS_AdjEM,Year
0,1,Kansas,30.23,115.8,85.5,67.3,0.040,12.66,107.4,94.7,9.58,2020
1,2,Gonzaga,26.95,121.3,94.4,71.9,0.050,2.42,103.5,101.0,-2.09,2020
2,3,Baylor,25.49,113.5,88.1,66.2,0.016,10.20,106.4,96.2,1.38,2020
3,4,Dayton,24.93,119.1,94.1,67.6,0.002,2.74,104.1,101.3,-0.74,2020
4,5,Duke,24.62,115.7,91.1,72.0,-0.009,7.28,106.0,98.7,2.60,2020


In [6]:
df_2020.to_csv(f'Data/kenPom_2020.csv', index=False)